In [ ]:
'''
!pip install streamlit pandas torch transformers langchain
!pip install pyngrok
!pip install langchain_community
'''

In [4]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Load the model and tokenizer
model_path = "bert_model_small.pth"
model = BertForSequenceClassification.from_pretrained('prajjwal1/bert-tiny', num_labels=4)
model.load_state_dict(torch.load(model_path))
tokenizer = BertTokenizer.from_pretrained('prajjwal1/bert-tiny')
model.eval()

config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

c:\Users\91963\anaconda3\envs\NLP_Summer\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\91963\.cache\huggingface\hub\models--prajjwal1--bert-tiny. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-12, e

In [8]:


# Load the BERT model and tokenizer
@st.cache_resource
def load_model():
    model_path = "bert_model_small.pth"
    model = BertForSequenceClassification.from_pretrained('prajjwal1/bert-tiny', num_labels=4)
    model.load_state_dict(torch.load(model_path))
    tokenizer = BertTokenizer.from_pretrained('prajjwal1/bert-tiny')
    return model, tokenizer

model, tokenizer = load_model()
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)

# Load the LangChain model
@st.cache_resource
def load_langchain_model():
    return OpenAI(model_name="text-davinci-003", api_key="your_openai_api_key")

langchain_model = load_langchain_model()

# Function to categorize responses
def categorize_response(response):
    labels = {'LABEL_0': 'AC', 'LABEL_1': 'PC', 'LABEL_2': 'TC', 'LABEL_3': 'NC'}
    result = classifier(response)
    return labels[result[0]['label']]

# Function to generate email content
def generate_email_content(name, concerns_label, anything_else_label, response):
    prompt = (
        f"Generate a short email for the respondent named {name} based on their categorized "
        f"concerns ({concerns_label}) and anything else ({anything_else_label}). Their response was: {response}"
    )
    email_content = langchain_model(prompt)
    return email_content

# Function to send email
def send_email(to_email, subject, body):
    from_email = "your_email@example.com"
    password = "your_email_password"
    
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] = from_email
    msg['To'] = to_email
    
    with smtplib.SMTP_SSL('smtp.example.com', 465) as server:
        server.login(from_email, password)
        server.sendmail(from_email, to_email, msg.as_string())

# Streamlit app
st.title('Survey Response Categorizer and Email Sender')

uploaded_file = st.file_uploader("Choose a CSV file", type="csv")
if uploaded_file is not None:
    df = pd.read_csv(uploaded_file)
    st.write(df.head())

    if st.button('Categorize and Send Emails'):
        for index, row in df.iterrows():
            concerns_label = categorize_response(row['concerns'])
            anything_else_label = categorize_response(row['anything else'])
            
            email_body = generate_email_content(
                row['name'], concerns_label, anything_else_label, row['concerns']
            )
            
            send_email(row['email'], "Your Survey Response Categorization", email_body)
            st.write(f"Email sent to {row['email']}")

        st.success("Emails sent successfully!")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\91963\anaconda3\envs\NLP_Summer\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
